In [6]:
!pip install nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.7 MB/s eta 0:00:00


In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
import torch
print(torch.cuda.device_count())

1


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the fine-tuned model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("./fine_tuned_seamless")
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_seamless")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from nltk.translate.bleu_score import corpus_bleu
from tqdm import tqdm
import torch

# Language codes
src_lang = "hin"  # Hindi
tgt_lang = "tam"  # Tamil

# Read and process data
with open("Hindi_Tamil_Tourism.hi", "r", encoding="utf-8") as f:
    hindi_sentences = [s.strip() for s in f.readlines()[:5000]]  

with open("Hindi_Tamil_Tourism.ta", "r", encoding="utf-8") as f:
    tamil_sentences = [s.strip() for s in f.readlines()[:5000]]

# Batch processing parameters
batch_size = 25
num_batches = 200
total_samples = batch_size * num_batches

# Results storage
all_translated = []
all_references = []
batch_bleu_scores = []

# Process in batches with progress bar
for batch_idx in tqdm(range(num_batches), desc="Processing Batches"):
    start_idx = batch_idx * batch_size
    end_idx = start_idx + batch_size
    
    # Get current batch
    hindi_batch = hindi_sentences[start_idx:end_idx]
    tamil_batch = tamil_sentences[start_idx:end_idx]
    
    # Translate batch
    translated_batch = []
    for sentence in hindi_batch:
        input_text = f"[{src_lang}] {sentence}"
        inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
        with torch.no_grad():
            output_tokens = model.generate(**inputs, tgt_lang=tgt_lang)
        translated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
        translated_batch.append(translated_text)
    
        # Calculate batch BLEU
    references = [[ref] for ref in tamil_batch]  # Wrap each reference in list
    batch_bleu = corpus_bleu(references, translated_batch)  # Direct float value
    batch_bleu_scores.append(batch_bleu)
    
    # Store results
    all_translated.extend(translated_batch)
    all_references.extend(tamil_batch)

    # Optional: Print batch progress
    tqdm.write(f"Batch {batch_idx+1}/{num_batches} - BLEU: {batch_bleu:.4f}")

# Calculate overall BLEU
overall_references = [[ref] for ref in all_references]
overall_bleu = corpus_bleu(overall_references, all_translated)

# Print results
print("\nBatch-wise BLEU Scores:")
for i, score in enumerate(batch_bleu_scores):
    print(f"Batch {i+1}: {score:.4f}")

print(f"\nOverall BLEU Score for {total_samples} samples: {overall_bleu:.4f}")

Processing Batches:   0%|▏                                    | 1/200 [00:25<1:23:02, 25.04s/it]

Batch 1/200 - BLEU: 0.6058


Processing Batches:   1%|▎                                    | 2/200 [00:51<1:25:10, 25.81s/it]

Batch 2/200 - BLEU: 0.4801


Processing Batches:   2%|▌                                    | 3/200 [01:15<1:21:58, 24.97s/it]

...

Batch 198/200 - BLEU: 0.5918


Processing Batches: 100%|██████████████████████████████████▊| 199/200 [1:14:52<00:25, 25.50s/it]

Batch 199/200 - BLEU: 0.6377


Processing Batches: 100%|███████████████████████████████████| 200/200 [1:15:15<00:00, 22.58s/it]


Batch 200/200 - BLEU: 0.6008

Batch-wise BLEU Scores:
Batch 1: 0.6058
Batch 2: 0.4801
Batch 3: 0.5153
Batch 4: 0.4824
Batch 5: 0.5807
Batch 6: 0.5248
Batch 7: 0.2587
Batch 8: 0.3148
Batch 9: 0.5687
Batch 10: 0.5494
Batch 11: 0.5679
Batch 12: 0.5710
Batch 13: 0.4650
Batch 14: 0.5904
Batch 15: 0.5820
Batch 16: 0.5178
Batch 17: 0.5154
Batch 18: 0.5193
Batch 19: 0.4960
Batch 20: 0.5348
Batch 21: 0.5323
Batch 22: 0.5218
Batch 23: 0.5450
Batch 24: 0.3233
Batch 25: 0.2270
Batch 26: 0.2372
Batch 27: 0.3126
Batch 28: 0.2634
Batch 29: 0.2400
Batch 30: 0.2409
Batch 31: 0.3108
Batch 32: 0.2787
Batch 33: 0.2597
Batch 34: 0.3145
Batch 35: 0.2649
Batch 36: 0.2655
Batch 37: 0.2550
Batch 38: 0.2992
Batch 39: 0.2633
Batch 40: 0.2588
Batch 41: 0.2977
Batch 42: 0.3012
Batch 43: 0.2767
Batch 44: 0.2292
Batch 45: 0.2369
Batch 46: 0.2493
Batch 47: 0.2658
Batch 48: 0.2504
Batch 49: 0.2607
Batch 50: 0.2222
Batch 51: 0.2455
Batch 52: 0.2455
Batch 53: 0.2517
Batch 54: 0.2542
Batch 55: 0.2667
Batch 56: 0.2480
Bat